# Απαλλακτική εργασία ΤΕΔεδομένων
Κωνσταντίνος Σκορδούλης 1115 201600155

In [1]:
import pandas as pd
import os

Create our dataframes

In [2]:
test_name = './data/impermium_verification_labels.csv'
train_name = './data/train.csv'

#Train_df = pd.read_csv(train_name, sep=',', usecols=['Insult','Comment'] , encoding ='utf-8')
#Test_df1 = pd.read_csv(test_name, sep=',', usecols=['id','Insult','Comment'], encoding='utf-8') # with labels

Train_df = pd.read_csv(train_name, sep=',', usecols=['Insult','Comment'])
Test_df1 = pd.read_csv(test_name, sep=',', usecols=['id','Insult','Comment']) # with labels

True_labels = Test_df1['Insult']  #used later for accuracy
Test_df = Test_df1[['id','Comment']] #without the labels

# Clearing our Comments

Για το data ccleaning κάναμε τα εξής βήματα:

1) BeautifulSoup().get_text(strip=True) ==> remove html elements

2) Remove mentions ( δηλαδή @user) => https://stackoverflow.com/questions/54733828/remove-twitter-mentions-from-pandas-column

3) Remove urls (starting with either http or https)

4) eval() ==> Χρησιμοποίησα την eval() για να καταπολεμήσω τα προβλήματα που είχα με τα \uxxx (δηλαδή τα unicode code points , τα μετατρέπω στους αντίστοιχους χαρακτήρες)

5) unidecode() . External βιβλιοθήκη, προπσθαεί να βρεί τον ισοδύναμο ascii χαρακτήρα (για να μπορέσουμε να κάνουμε σωστά την δουλειά μας αργότερα)

6) Δυστυχώς δεν βρήκα κάποιο αυτόματο τρόπο να μετατρέψω τα <b>\xyy</b> , οπότε έψαχνα τον utf8 πίνακα και έκανα <b>χειροκίνητα</b> τις αλλαγές (υποψιάζομαι ότι υπάρχει τρόπος το <b>\xyy</b> να γίνει <b>\uxxx</b> και απο κεί να το άφηνα στην eval()
    
7)Clean special chars, punctuations (replace with whitespace) and lowercase everything
 ==> https://stackoverflow.com/questions/34860982/replace-the-punctuation-with-whitespace

8) Έκανα ένα τελευταίο καθαρισμό από \n,\t που μπορεί να είχαν κρυφτεί στο κειμενό μου

<b> Fin </b>

In [5]:
from bs4 import BeautifulSoup
import re

import unicodedata as ud
from unidecode import unidecode

import string

In [6]:
regex_url = r'https?://[A-Za-z0-9./]+'

regex_mention = r'@[A-Za-z0-9]+'


def clean_comments(dataframe):
    
    df1 = dataframe # a copy
    dirty_comments = df1['Comment'].to_list()
    
    clean_list = []
    
    for x in dirty_comments: #for each dirty comment       
        
        ####### 1) BeautifulSoup().get_text(strip=True) ==> remove html elements
        soup = BeautifulSoup(x,'lxml')
        clean1 = soup.get_text(' ',strip=True) #get clean text (human_readable text)
        
        ###### 2) Remove mentions
        clean2 = re.sub(regex_mention,'',clean1)
        
        ##### 3) Remove urls
        clean3 = re.sub(regex_url,'', clean2)  
        
        ###### 4) eval()
        clean3 = clean3.replace('"','') #remove double quotes? from start and finish ==> else problem with eval()
        clean4 = eval("u\"" + clean3 + "\"") #force process the \xa0 \uxxx  unicode code points into their characters


        ###### 5) Unidecode() ==> find approximate ascii char
        clean5 = unidecode(clean4)
        
        ###### 6) Some last chars to delete \xc2\xa0 etc        
        clean6 = re.sub(r'\\xc2\\xa0',' ',clean5) #simple space
        clean6 = re.sub(r'\\xe2\\x80\\x99','',clean6) #this is a right single quote => remove it
        clean6 = re.sub(r'\\xe2\\x80\\xa6',' ',clean6) # this is ...
        clean6 = re.sub(r'\\xc3\\xb3','o',clean6) #this is a weird char for "o"
        clean6 = re.sub(r'\\xe2\\x80\\x9c','',clean6) #delete double quotation marks
        clean6 = re.sub(r'\\xc2\\xa3',' ',clean6) #delete pound symbol
        clean6 = re.sub(r'\\xe2\\x80\\x9d','',clean6) #delete double quotation marks
        clean6 = re.sub(r'\\xe2\\x80\\x98','',clean6) #delete single quotation marks
        clean6 = re.sub(r'\\xf0\\x9f\\x91\\x99',' ',clean6) # emoji delete        
        clean6 = re.sub(r'\\xf0\\x9f\\x8d\\xbb',' ',clean6) # emoji delete
        clean6 = re.sub(r'\\xe2\\x80\\x93',' ',clean6) # _ delete
        clean6 = re.sub(r'\\xe2\\x80\\x94',' ',clean6) # another _ delete
        clean6 = re.sub(r'\\xe3\\x80\\x80',' ',clean6) #just a space
        clean6 = re.sub(r'\\xc2\\xa7',' ',clean6) #section sign => delete
        clean6 = re.sub(r'\\xe2\\x80\\x89',' ',clean6) # space again
        clean6 = re.sub(r'\\xef\\xbb\\xbf',' ',clean6) # delete utf-sig encryption (BOM)
        clean6 = re.sub(r'\\xc8\\x9b','t',clean6) # weird character for (t)
        clean6 = re.sub(r'\\xc8\\x99','s',clean6) # weird character for (s)
        clean6 = re.sub(r'\\xc4\\x83',' ',clean6) # weird character for (a)
        clean6 = re.sub(r'\\xe2\\x9c\\x8c\\xf0\\x9f\\x94\\xab',' ',clean6) #delete emoji
        
        clean6 = re.sub(r"\\'",'',clean6) #remove the (\')
        clean6 = re.sub(r'\\n',' ',clean6)
        clean6 = re.sub(r'\\t',' ',clean6)

        
        # 7) Clean special chars, punctuations (replace with whitespace) and lowercase everything
        # https://stackoverflow.com/questions/34860982/replace-the-punctuation-with-whitespace
        
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
        clean7 = clean6.translate(translator)
        clean7 = clean7.lower() # finally lowercase the string

        # 8) Delete \n, \t  (in case they show up )
        clean8 = clean7.replace('\n',' ')
        clean8 = clean8.replace('\t', ' ')
        
        print(clean8)
        clean_list.append(clean8)
    
    #end for
    s1 = pd.Series(clean_list)
    df1['Comment']=s1
    
    return df1       

Clean the training set

In [7]:
clean_train = clean_comments(Train_df)

you fuck your dad 
i really don t understand your point   it seems that you are mixing apples and oranges 
a majority of canadians can and has been wrong before now and will be again   unless you re supportive of the idea that nothing is full proof or perfect so you take your chances and if we should inadvertently kill your son or daughter then them s the breaks and we can always regard you as collateral damage like in wartime   and sorry  but the cheques in the mail  
listen if you dont wanna get married to a man or a women dont do it  what would it bother you if gay people got married stay in your lane do you let them do them  and your god is so nice but quick to judg if your not like him  thought you wasnt suppose to judge people 
cac ban xuong duong bieu tinh 2011 co on hoa khong    cac ngu dan ngoi cui dau chiu nhuc co on hoa khong   cac nong dan giu dat o van giang  can tho co on hoa khong                     rot cuoc duoc gi  thi chung ta da biet   ai cung yeu chuong hoa binh  n

jolie is my qurl  excited as hell to see this movie  too bad its 2 years away   fuck
you are a child 
libero ha per vicedirettore un signore che ha pubblicato l audio di una telefonata privata con un parlamentare  crosetto  libero fa campagne contro le intercettazioni e poi pubblica telefonate private  quel testa di c e andato a milano diceva crosetto del suo boss   il direttore di libero si e fatto un attentato da solo  o forse e stata la guardia del corpo recidiva  pansa  il megagiornalista  su libero ha scritto  e mai smentito  che enrico berlinguer ha telefonato a curzi per raccomandare la figlia  nel 1991  da morto  questo e libero  che si prendano freda 
hang on  eduardo  what s your home address  i ll come forcefully kick  your door down  take over your house  if you try and kick me out  i ll  seriously physically harm you  because as you pointed out  right to self  defence   i hope you realise how retarded your argument is  truely 
 little girl  you  you of all people make that

who is the real idiot  dogtwon or me   you have to pick one you know  everybody can t be the stupidest person you talk to unless you talk to yourself  you asked for this with your lovely comments on how stupid everyone is that didn t vote for ron paul what did he wind u with   last place and about 3 percent of the vote   so 97   of the other primary voters are all stupid morons and you are enlightened  
who gives a rat s rump what geithner says or thinks  really  he has proven to be an incompetent obama puppet and his failures are stacking up faster than pancakes when rosie o donnell raids her local waffle house    this guy is going down faster than janet napolitano when elena kagan comes to visit   sorry for the visual  it made me puke too  
god is not a republican  nor an american  nor a conservative
we  remain happy with our choices   have been quite sure that you are not settled 
there is nothing equal about a hetrosexual marriage and a gay marriage  you can t compare apples to ora

well then the op would learn that she cannot diagnose herself with aspergers  and that self diagnosis is a bad thing leading to people whom act stupidly because they think they have some disorder disease that in actuality  they don t 
you just went full retard   never go full retard 
somehow  i feel that pakistan is right in the middle of all of this garbage  yet  the united states keeps supplying them with arms and money  no sense to this at all 
you act like you faced a good team  you played like crap against another crappy team 
he still played  he didn t say that much  and obviously the heat reserves are going to lose that either way  it is when lebron and wade are playing this post season that the celtics need to worry 
that sounds familiar  mom picking subscriptions with no veto power to stop her 
 lancaster pa mah friend  wez ghetto
he did the right thing  too bad someone didn t do the same thing to sandusky 
eish wenger did a stupid mistake but he will never admit it anyway    

what kills me is if you disagree with me that you are a racist 
you don t wear your brain bucket     you only hurt you and your family  you become impaired you have the ability to hurt everyone 
he s not an isolationist  he just does not want a foreign policy that ends in america stomping all over the world like babylonians    only ron paul will defend israel when they blow up iran s nuclear sites just like only ron paul defended israel when they blew up iraq s nuclear sites   ron paul is hardly an isolationist  he just recognizes a lot of tough truths about america s foreign policy that the self righteous warmongers just can t seem to swallow  
oh pops you must really really be enamoured with bill you naughty boy you
we could start a media blitz like they did with the drug campaign   you know  the one        this is your brain  this is         we could show pictures of pelosi and say       this is your bomb vest       heres your reward  when you blow yourself up 
nice use of vulgarity

...

as always you said it succinctly  it is santorums continued integrity that makes him worthy of a conservative vote  should he lose because he did not compromise his values  it says more about our country and christianity in the west than it does rick  lets pray america is not so far gone that todays voter puts more stock in the willingness to do whatever it takes than the willingness do what is right regardless the cost 
i assume that you are trolling  i can t see any other reason for your post 
 i think hallucinations could make for a very interesting feb of writing for you 
well nothing is torn so i don t get how much more injured someone can be  i think the rangers are just being really cautious with feliz
grrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr i hate that motherfuck her dad need  to be burned at stake
gee whiz ryan is your dik that small  maybe you should invest in some of that dik extender instead of a new computer  you are nothing but a keyb

you r getting too old  before you hit the post botton  see how many posts there already  open them up  then post yours 
burr headed  purple lipped kenyan       are you a human or a right wing asshole   go back under your bridge 
guess what  no one gives a fuck about you  get off snsd articles 
great post dan  what an amazing story   you did the right thing   i know i have an experience just can t think off the top of my head    i think what you did was pure kindness   some people may not have stopped and even talked to her   im sure where ever she is i know she is telling the same story about how an amazing guy called and helped her   she may be wondering how you are   i guess the best part of life is not getting credit but just doing good deeds and just moving on   very awesome dan 
i know more intelligent fans of the timberwolves than intelligent fans of the heat  those people are clueless for the most part 
lizzaronna  not only does your ignorance shine through per your ignorant com

you re clearly retarded 
sure they are tommy one less demwad
being gay is absolutely a choice   if it were not a choice  how do you explain the thousands of people who were in that lifestyle and have rejected it and become heterosexual 
the general population may be  but i think that the people in the nar know exactly what is in there  and are advocating for it  these are the same people who think gays should be executed  atheists should have their citizenship revoked  and that the first amendment means you have the freedom to choose what type of christian you want to be  they say they want to return us to a christian nation and that we need to base our laws on old testament biblical law  go back and look at some of the stuff bryan fischer  mike huckabee  michele bachmann  and sarah palin have both said and endorsed  look at some of the statements rick perrys advisers have made  and things jim demint has said  they feel they have been commanded to take over government  the entertainmen

let them eat cake     words that lit the french revolution   what words will americans have to hear   eat the rich   
where s all the ron paul trolls  to defend this piece of shit 
emontional restitution sounds like you tried to cash in but failed
i like his films    put more fighting in vin      maybe team up with jason statham  transporter 
i love how yo are a complete moron  and you havvee to be a complete retard to hate naruto  like seriously man  can you even form proper sentences  well  obviously not 
you are a complete idiot   not even a partial 
da fuck this nigga talkin bout  everything this hoe was doing was turning me on      this nigga gay
irony is not in most conservative s vocabulary 
he killed himself like a little sissy   good riddance scumbag
hello wshh it is i again nastasha  for the record no i do not strip  i m on south beach ocean drive leaving a pool party  get over yourselves  i love me  and at the end of the day u can think what u want  an opinion is like an ass

and nobody partially gives a fuck about you or your garbage state
and what are you going to do tough guy 
you do realize  big 12  sec  b1g or pac teams would have to lose 1 2 games for an acc team to make the ncg  right 
take your meds
my favorite part was show killing vince with a shot of air 
so in your perverted mind if someone posts something about obama that is discredited  it is ok for you to post the unmitigated bullshit that you do   whhhhhhhhaaaaaaaaaaaaaa ha ha ha
to be fair to troyzan  he didn t start alienating people until it was 100  clear that he was going home the next time he didn t have immunity   that s when he started acting like a dick  doing the whole this is my island  shit when he won  etc   before that  he was working the social game just fine 
president romney     it has a nice ring to it    
you are an idiot and doubt very seriously you have any proof to back such a ridiculous statement
 which one there s 2    
campy is sorta similar to compy  which brings us

Clean the Testing set

In [8]:
clean_test = clean_comments(Test_df)

like this if you are a tribe fan
you re idiot                       
i am a woman babs  and the only war on women i see is coming from jackazzes like you   i don t need your protection or your ignorant rhetoric masquerading as representing my best interests   
wow   you benefitted so many wins this year from his bat nice stupid
haha green me red you now loser whos winning now moron
 me and god both hate faggots   what s the difference between a fag and a refrigerator  refrigerators don t fart when you put the meat in 
oh go kiss the ass of a goat    and you dummycraps insult veterans    even your coward incompetent chief     everyone of you are the enemy    and you ll all fall     and hard too boy 
not a chance kid  you re wrong 
on some real shit fuck live jasmin   
ok but where the hell was it released you all copy articles from each pther but did anyone do the fucking search to tell us where the fuck was it released fuck it 
  has found our slice of the internets   fuck you 
 hahaha

you are a moron  get off the guys dick
what are you talking about   totally off 
firstly  go back to skool  you really need to learn how to spell before you start calling someone retarded and i m sorry you have been sadly misinformed i chose to walk away from the drama and lies  if the locks were changed  it was a waste of time i threw my key away when i made the choice to walk away  and if you had the balls to face me  you would not have hidden like a coward when they tried to serve you  or you would have shown up and backed your girl  
omg   pamela   you are so right    
the knicks  wtf are you smoking 
what are you talking about   
only if you were living in the other half
are you sure this won t induce epilepsy blurred vision both     
hey  einstein  perhaps you should have read and understood what you quoted  since it clearly outlines a war room to blunt any movement post dnc  not during   is it really shocking that the gop would have a war room in nc  a swing state    reading is 

you always are   
u dad already ashamed of u cuz he dnt know if u really his son cuz your mom was sucking and fuc king the crew back in da days  lmao   now u grew up to hate black men wit money cuz u know dsts what your mom is looking for  lol  tell her i ll pass up on dat a ss cuz it got to many miles on it howeva she can give me some slow head   lol  lol  lol   
well maybe if you didn t let yourself go and pack on all those pounds 
keep digging  your hole isn t quite deep enough yet    
he won t won anything all time with arsenal   i bet you re one guys that saying it arsenal will finish outside top 4th  i hope u don t change your fucking brain when we win titles   u brainless kid
   no they are your hero because your stones still haven t dropped and you re too much of a girly man to pull the trigger  put down your computer and get involved nerd boy  don t be a pathetic loser your whole life  strap up or s tf u 
your comments are as retarded as you are  typical united hater
idiot  go

your use of  leadership  betrayed you 
you sound stupid 
vinnnie   you are the best rapper       yout german fans   
nope  not just you 
the exponential growth of government is destroying the economy   the democrats  poisonous demagoguery has created the vast prison system  the welfare rolls  and a shrinking middle class   propaganda has the greatest effect on the least intelligent   the class warfare rhetoric  the myth of the rich stealing from the poor  and the raised clinched fist has  sent society s dumb into the streets to take their fair share   the net result of this is terror  murder  rape  pillage  and plunder   also  these most unintelligent of the nation become a burden for the nation to cage   the those of our society with a slightly higher iq  accept the demented life of the welfare queen   the life of the welfare class is one of drug addiction  sexual depravity  abortion as contraceptive  obesity  savage neighborhoods  and roach infested squalor   the next level of the un

get off your knees your embarrassing yourselves 
 you are proving yet again that liberals are masters of doublethink   
are you his employer        see how this works 
i m guessing english is not your first language  he was being sarcastic 
check your own username   doofus  and your spelling 
you  are crazyy
 spent more than all others combined  are you insane 
what a loser you are 
anna you are so talented  
fuck you mikal  don t laugh at something i say you stupid little punk ass bitch  no one likes you  pussy 
who the fuck gives a shit peanut what forbes says  that s like if fucking people magazine wrote up an opinion paper about diablo3  it wouldn t be worthy of wiping my ass with 
ssshhhhhh you are being logical
you must be related  to the nigger above  live in the same trailer park do ya   
fucking spoiled rich fuckers have no clue what it is to make a dollar  go and work for a living and see if you burn a 100k bag  hell you wouldnt even be burning a 50 dollar bag  that is not ar

he also taught us to stay the fuck away from ice nine  
what are you waiting for 
akin you selfish sob  drop out of the race which you cannot possible win   take a back seat now and but support someone who is still untarnished   start rebuilding your reputation by doing what is best for the country   what in the hell is wrong with you    
fuck 
futballi ju ka met  ropt jav cifsha juve    yall a buch of fucking idiots
let s not play your idiotic game  polish nationals who endorsed nazi doctrine were employed by the nazis  that does not make a joint venture of any sort 
oh boohoo  you people need to suck it up  she looks hot  she s not naked  have you seen what rihanna and all those crappy singers wear  like it said in the article  if you lazy asses bothered to read it  this is just for fun  she s not gonna start dressing like that all of a sudden  she has an incredible voice and an incredible body  this isn t a plea for attention  if she wanted to be noticed she would start a fight with

don t let a few clowns ruin your board experience  we have our idiots too  remember 
you re too easy
you know  you are only supposed to pull for your team right   not the whole conference  
can i pay your loans off for you 
maybe he s recently resurrected from the dead   let s not be quick to judgement   you try knowing right from wrong when your stomach is empty and your skull is full of worms and maggotts    let me tell ya    it s not that easy   
del rio has been heavyweight champion before  so i don t know what you guys are talking about  he has had big wins  and has been champion  know what your talking about instead of sounding like idiots 
o wow cus eminem said it its the law of the land huh  an im da idiot  an since u wana quote eminem who da fuck raised hailey cus he didnt 
cricket  you re being taken for a ride by your party
you obviously don t understand sarcasm  
fuck me  plz   
    hey  hey  hey gop   baggers         time to pull out your secret weapon        bring back yo

you said oh shush lmaooo wut 
these niggas are hilarious   
you must be outta coke     
lady  are you kidding  yourself 
when one of your family members is murdered by someone high on cocaine    then you can voice your advocacy  trash 
whp the fuck googles a comment im not you bitch ass nigga stfu 
you would look super kawaii in this xx
this is from a white woman s perspective who has dated black men  educated black men  when black women only fuck with thugs in their early adulthood  then get knocked up  and having never given the nerdy black guys a chance because they were corny  meanwhile  us white women who appreciate brains and character over swag from our early stages of adulthood and hold onto these values throughout life  notice these rejected black men  so now  what you see in this video is a white woman who snatched up what black women rejected  life is a bitch  you mad  black women are failures are true womanhood  just saying  
romney s team pays more if you strap your dog to

C:\Users\kostantinos\anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Save our clean comments into new .csv files

In [9]:
train_name1 = r'./clean_train.csv'
test_name1 = r'./clean_test.csv'

In [10]:
Test_df1['Comment'] = clean_test['Comment']
Test_df1.to_csv(test_name1)

Train_df['Comment'] = clean_train['Comment']
Train_df.to_csv(train_name1)